Recommendation system

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split


from ast import literal_eval
import pandas as pd
import numpy as np

import pickle
import torch

from sentence_transformers import SentenceTransformer,util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
arxiv_data = pd.read_csv('/content/arxiv_data_210930-054931.csv', on_bad_lines='skip')

In [3]:
arxiv_data.head()

,terms,titles,abstracts
0,['cs.LG'],Multi-Level Attention Pooling for Graph Neural...,Graph neural networks (GNNs) have been widely ...
1,"['cs.LG', 'cs.AI']",Decision Forests vs. Deep Networks: Conceptual...,Deep networks and decision forests (such as ra...
2,"['cs.LG', 'cs.CR', 'stat.ML']",Power up! Robust Graph Convolutional Network v...,Graph convolutional networks (GCNs) are powerf...
3,"['cs.LG', 'cs.CR']",Releasing Graph Neural Networks with Different...,With the increasing popularity of Graph Neural...
4,['cs.LG'],Recurrence-Aware Long-Term Cognitive Network f...,Machine learning solutions for pattern classif...


In [4]:
arxiv_data.shape

(41026, 3)

In [5]:
arxiv_data.drop(columns=['terms', 'abstracts'], axis=1, inplace = True)

Sentence Transformer

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
sentences = arxiv_data['titles']

In [8]:
embeddings = model.encode(sentences)

In [9]:
embeddings

array([[ 0.06643405, -0.04954598,  0.06388085, ...,  0.00106302,
        -0.12156379, -0.06962775],
       [ 0.09212255, -0.07606938,  0.06572867, ..., -0.08565167,
        -0.09266547,  0.00725293],
       [-0.08162681,  0.02428931,  0.01888746, ...,  0.0080616 ,
        -0.0512953 , -0.05873995],
       ...,
       [ 0.00547755,  0.0694479 ,  0.09368725, ..., -0.08764996,
         0.02108502, -0.05680082],
       [-0.08659958,  0.02304513,  0.110962  , ..., -0.07064024,
        -0.00736168,  0.00995756],
       [-0.03520672, -0.08613261,  0.06290741, ...,  0.01816004,
        -0.05791019, -0.08266117]], dtype=float32)

In [10]:
embeddings.shape

(41026, 384)

Test

In [11]:
c = 0

for sentence, embedding in zip(sentences, embeddings):
  print("Sentence:", sentence)
  print("Embedding:", len(embedding))
  print("")
  c += 1
  if c == 10:
    break

Sentence: Multi-Level Attention Pooling for Graph Neural Networks: Unifying Graph Representations with Multiple Localities
Embedding: 384

Sentence: Decision Forests vs. Deep Networks: Conceptual Similarities and Empirical Differences at Small Sample Sizes
Embedding: 384

Sentence: Power up! Robust Graph Convolutional Network via Graph Powering
Embedding: 384

Sentence: Releasing Graph Neural Networks with Differential Privacy Guarantees
Embedding: 384

Sentence: Recurrence-Aware Long-Term Cognitive Network for Explainable Pattern Classification
Embedding: 384

Sentence: Lifelong Graph Learning
Embedding: 384

Sentence: Bayesian graph convolutional neural networks via tempered MCMC
Embedding: 384

Sentence: Understanding and Resolving Performance Degradation in Graph Convolutional Networks
Embedding: 384

Sentence: Edge-augmented Graph Transformers: Global Self-attention is Enough for Graphs
Embedding: 384

Sentence: Variational Graph Normalized Auto-Encoders
Embedding: 384



In [12]:
with open('embeddings.pkl', 'wb') as f:
  pickle.dump(embeddings, f)

with open('sentences.pkl', 'wb') as f:
  pickle.dump(sentences, f)

with open('rec_model.pkl', 'wb') as f:
  pickle.dump(model, f)

In [13]:
embeddings = pickle.load(open('embeddings.pkl', 'rb'))
sentences = pickle.load(open('sentences.pkl', 'rb'))
recc_model = pickle.load(open('rec_model.pkl', 'rb'))

In [14]:
a = "Attention is all you need"
myemb = recc_model.encode(a)
util.cos_sim(embedding, myemb)

tensor([[0.1495]])

In [15]:
def recommendation(input_paper, threshold=0.9):
    cosine_scores = util.cos_sim(embeddings, recc_model.encode(input_paper))

    top_similar_papers = torch.topk(cosine_scores, dim=0, k=20, sorted=True)

    papers_list = []
    seen_titles = set()

    for i in top_similar_papers.indices:
        title = sentences[i.item()]
        if title not in seen_titles:
            papers_list.append(title)
            seen_titles.add(title)

        if len(papers_list) == 10:
            break

    return papers_list

In [16]:
# usage = Attention is all you need
input_paper = input("enter paper title")
recommended_papers = recommendation(input_paper)

recommended_papers

enter paper titleAttention is all you need


['Attention that does not Explain Away',
 'Area Attention',
 'Pay Attention when Required',
 'Long Short-Term Attention',
 'Attention as Activation',
 'Invertible Attention',
 'Attention is Not All You Need: Pure Attention Loses Rank Doubly Exponentially with Depth',
 'Bottom-up Attention, Models of',
 'Visual Attention: Deep Rare Features',
 'On the Regularity of Attention']

In [17]:
# usage = (Review of deep learning: concepts, CNN architectures, challenges, applications, future directions)
input_paper = input("enter paper title - ")
recommended_papers = recommendation(input_paper)

recommended_papers

enter paper title - Review of deep learning: concepts, CNN architectures, challenges, applications, future directions


['A Review of Deep Learning with Special Emphasis on Architectures, Applications and Recent Trends',
 'Review of Deep Learning',
 'Deep Convolutional Neural Networks: A survey of the foundations, selected improvements, and some current applications',
 'A Survey of the Recent Architectures of Deep Convolutional Neural Networks',
 'A Survey of Convolutional Neural Networks: Analysis, Applications, and Prospects',
 'Deep Learning for Medical Image Processing: Overview, Challenges and Future',
 'A Survey on Understanding, Visualizations, and Explanation of Deep Neural Networks',
 'Training Deep Architectures Without End-to-End Backpropagation: A Brief Survey',
 'A Non-Technical Survey on Deep Convolutional Neural Network Architectures',
 'Object Detection with Deep Learning: A Review']